In [2]:
import json

# GPT-4 logs

### Minimal prompt

In [14]:
mix_benchmarks_file1 = json.load(open("gpt_4/minimal/old_mix/loopy_2023_08_27_02_50_01/final_rechecked_re_filtered.json", "r", encoding="utf-8"))
mix_benchmarks_file2 = json.load(open("gpt_4/minimal/old_mix/loopy_2023_08_17_03_55_30/final_rechecked_re_filtered.json", "r", encoding="utf-8"))
diff_file = json.load(open("gpt_4/minimal/diff_files/loopy_2023_10_28_04_19_25/final.json", "r", encoding="utf-8"))
code2inv_file = json.load(open("gpt_4/minimal/code2inv/loopy_2023_10_31_09_39_01/final.json", "r", encoding="utf-8"))

In [23]:
combined_json = {
    "params" : {
        "config_file": "expt3.yaml"
    },
    "logs" : [],
    "files_to_rerun" : []
}

In [24]:
for log in zip(mix_benchmarks_file1["logs"], mix_benchmarks_file2["logs"]):
    if "completions" not in log[0] or "completions" not in log[1]:
        combined_json["files_to_rerun"].append(log[0]["file"])
        continue
    instance_json = {
        "file" : log[0]["file"],
        "benchmark_code" : log[0]["benchmark_code"],
        "completions" : log[0]["completions"] + log[1]["completions"],
        "invariants" : log[0]["invariants"] + log[1]["invariants"],
        "code_with_combined_invariants" : "",
        "checker_output" : None,
    }

    combined_json["logs"].append(instance_json)

In [25]:
for log in diff_file["logs"]:
    if "completions" not in log:
        combined_json["files_to_rerun"].append(log["file"])
        continue
    instance_json = {
        "file" : log["file"],
        "benchmark_code" : log["benchmark_code"],
        # "completions" : log["completions"],
        "invariants" : log["annotation_blocks"],
        "code_with_combined_invariants" : log["code_with_combined_annotations"],
        "checker_output" : log["checker_output_for_combined_annotations"],
    }
    completions = []
    for completion in log["completions"]:
        completions.append({
            "invariants": completion["annotations"],
            "success": completion["checker_output_for_annotations"],
            "checker_message": completion["checker_message_for_annotations"],
        })
        if "checker_output_after_prune" in completion:
            completions[-1]["success_after_prune"] = completion["checker_output_after_prune"]
            completions[-1]["pruned_code"] = completion["code_after_prune"]
    instance_json["completions"] = completions
    if "code_after_prune" in log:
        instance_json["code_after_combine_and_prune"] = log["code_after_prune"]
        instance_json["checker_output_after_combine_and_prune"] = log["checker_output_after_prune"]

    combined_json["logs"].append(instance_json)

In [26]:
for log in code2inv_file["logs"]:
    if "completions" not in log:
        combined_json["files_to_rerun"].append(log["file"])
        continue
    instance_json = {
        "file" : log["file"],
        "benchmark_code" : log["benchmark_code"],
        # "completions" : log["completions"],
        "invariants" : log["annotation_blocks"],
        "code_with_combined_invariants" : log["code_with_combined_annotations"],
        "checker_output" : log["checker_output_for_combined_annotations"],
    }

    completions = []
    for completion in log["completions"]:
        if not "checker_output_for_annotations" in completion:
            completions.append({
                "checker_output_for_annotations" : completion["success"],
                "error" : completion["error"],
                "llm_output" : completion["llm_output"],
            })
            continue
        completions.append({
            "invariants": completion["annotations"],
            "success": completion["checker_output_for_annotations"],
            "checker_message": completion["checker_message_for_annotations"],
        })
        if "checker_output_after_prune" in completion:
            completions[-1]["success_after_prune"] = completion["checker_output_after_prune"]
            completions[-1]["pruned_code"] = completion["code_after_prune"]

    combined_json["logs"].append(instance_json)

In [27]:
with open("gpt_4/combined_logs_m0_prompt.json", "w", encoding="utf-8") as f:
    json.dump(combined_json, f, indent=4, ensure_ascii=False)

### With nudges prompt

In [9]:
diff_file = json.load(open("gpt_4/with_nudges/diff_files/loopy_2023_10_27_10_53_58/final.json", "r", encoding="utf-8"))
code2inv_file = json.load(open("gpt_4/with_nudges/code2inv/loopy_2023_10_03_02_16_01/final_rechecked.json", "r", encoding="utf-8"))
mix_benchmarks_file1 = json.load(open("gpt_4/with_nudges/old_mix/loopy_2023_08_25_13_54_09/final_rechecked_re_filtered.json", "r", encoding="utf-8"))
mix_benchmarks_file2 = json.load(open("gpt_4/with_nudges/old_mix/loopy_2023_08_13_23_52_42/final_rechecked_rechecked_re_filtered.json", "r", encoding="utf-8"))

In [16]:
combined_json = {
    "params" : {
        "config_file": "expt2.yaml"
    },
    "logs" : [],
    "files_to_rerun" : []
}

In [17]:
for log in diff_file["logs"]:
    if "completions" not in log:
        combined_json["files_to_rerun"].append(log["file"])
        continue
    instance_json = {
        "file" : log["file"],
        "benchmark_code" : log["benchmark_code"],
        # "completions" : log["completions"],
        "invariants" : log["annotation_blocks"],
        "code_with_combined_invariants" : log["code_with_combined_annotations"],
        "checker_output" : log["checker_output_for_combined_annotations"],
    }
    completions = []
    for completion in log["completions"]:
        if not "checker_output_for_annotations" in completion:
            completions.append({
                "checker_output_for_annotations" : completion["success"],
                "error" : completion["error"],
                "llm_output" : completion["llm_output"],
            })
            continue
        completions.append({
            "invariants": completion["annotations"],
            "success": completion["checker_output_for_annotations"],
            "checker_message": completion["checker_message_for_annotations"],
        })
        if "checker_output_after_prune" in completion:
            completions[-1]["success_after_prune"] = completion["checker_output_after_prune"]
            completions[-1]["pruned_code"] = completion["code_after_prune"]
    instance_json["completions"] = completions
    if "code_after_prune" in log:
        instance_json["code_after_combine_and_prune"] = log["code_after_prune"]
        instance_json["checker_output_after_combine_and_prune"] = log["checker_output_after_prune"]

    combined_json["logs"].append(instance_json)

In [18]:
for log in code2inv_file["logs"]:
    if "completions" not in log:
        combined_json["files_to_rerun"].append(log["file"])
        continue
    instance_json = {
        "file" : log["file"],
        "benchmark_code" : log["benchmark_code"],
        "completions" : log["completions"],
        "invariants" : log["invariants"],
        "code_with_combined_invariants" : log["code_with_combined_invariants"],
        "checker_output" : log["checker_output"],
    }
    if "code_after_combine_and_prune" in log:
        instance_json["code_after_combine_and_prune"] = log["code_after_combine_and_prune"]
        instance_json["checker_output_after_combine_and_prune"] = log["checker_output_after_combine_and_prune"]

    combined_json["logs"].append(instance_json)

In [19]:
for log in zip(mix_benchmarks_file1["logs"], mix_benchmarks_file2["logs"]):
    if "completions" not in log[0] or "completions" not in log[1]:
        combined_json["files_to_rerun"].append(log[0]["file"])
        continue
    instance_json = {
        "file" : log[0]["file"],
        "benchmark_code" : log[0]["benchmark_code"],
        "completions" : log[0]["completions"] + log[1]["completions"],
        "invariants" : log[0]["invariants"] + log[1]["invariants"],
        "code_with_combined_invariants" : "",
        "checker_output" : None,
    }

    combined_json["logs"].append(instance_json)

In [20]:
with open("gpt_4/combined_logs_m1_prompt.json", "w", encoding="utf-8") as f:
    json.dump(combined_json, f, indent=4, ensure_ascii=False)

# GPT-3.5-Turbo logs